## 1. 필요한 라이브러리 불러오기

## 2. Reading Train and Test Datasets

/var/folders/1_/1w14611n7zj6b7_3h1l8lqm80000gn/T/ipykernel_14958/3920746870.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/var/folders/1_/1w14611n7zj6b7_3h1l8lqm80000gn/T/ipykernel_14958/3920746870.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

## 3. Preprocessing

## 4. Cross-Validation

In [4]:
# 최적의 모델로 예측
best_gb_clf = grid_search.best_estimator_
y_pred = best_gb_clf.predict(X_test)

# 모델 성능 측정
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")


AttributeError: 'GradientBoostingClassifier' object has no attribute 'estimators_'

## 5. Submission